<a href="https://colab.research.google.com/github/tjturnage/radar/blob/main/Plot_NEXRAD_mosaics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="+2" color="blue">Plot NEXRAD Reflectivity Mosaics</font>  
<font color="blue">*v1.03 - updated 2/18/2024*</font>

*T.J. Turnage - NWS Grand Rapids, MI -- thomas.turnage@noaa.gov*  
<br />
<hr>
<br/>


This allows the user to plot nexrad mosaic reflectivity images described at:
https://mesonet.agron.iastate.edu/docs/nexrad_mosaic/
<br />
<br />
The user can define the following:
- whether to use high or low resolution plots (low-res goes back to 1995)
- form-based selection of time range to plot
- time zone to include in the label
- plotting interval (that is, make a plot for every 5, 10, 15, etc., minutes)
- lat/lon box containing the plot
- one of two available reflectivity colormaps (additional maps available on request)
- which cities to plot (if any) and where to place the labels
- whether to plot major roads
- whether to plot county outlines
- figure size and plot aspect ratio (usually don't need to change)
- whether to create an animated gif
<br/> <br/>

A file called "radar_images.zip" will be created. This file can be downloaded and then unzipped on your local machine to extract the image files.

<br />

<font size="+1">*v1.03 updates (18-Feb-2024)*</font>
*   fixed color map selection error  

<br />
<hr>

In [1]:
# @markdown <font color="blue"><h2>Select the type of plot</h2></font>
# @markdown <i><font color="blue"><h4>low res is good for multi-state plots!</h4></font></i>


plot_type = "high res (available since Nov2010)" # @param ["low res (available since 1995)", "high res (available since Nov2010)"]

file_type = 'n0r'
if plot_type == "high res (available since Nov2010)":
  file_type = 'n0q'

# @markdown <br><font color="blue"><h2>Enter the start time (in UTC) for the plots</h2></font>
start_date = "2024-02-17" # @param {type:"date"}
start_date = start_date.replace('-', '')
start_hour = "10" # @param ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17,","18","19","20","21","22","23"]
start_minute = "55" # @param ["00", "05", "10", "15", "20", "25", "30", "35", "40", "45", "50", "55"]

# @markdown <br><font color="blue"><h2>Enter the end time (in UTC) for the plots</h2></font>
end_date = "2024-02-17" # @param {type:"date"}
end_date = end_date.replace('-', '')
end_hour = "11" # @param ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17,","18","19","20","21","22","23"]
end_minute = "00" # @param ["00", "05", "10", "15", "20", "25", "30", "35", "40", "45", "50", "55"]

if start_date < '20101113' and file_type == 'n0q':
  print("High res not available for this time! ...")
  print("... selecting low res instead ...")
  file_type = 'n0r'

# @markdown <br><font color="blue"><h2>Select time zone for labelling your figures</h2></font>
time_zone = "EST" # @param ["UTC", "EST", "EDT", "CST", "CDT", "MST", "MDT", "PST", "PDT"]
time_zone_dict = {"UTC": 0, "EST":5, "EDT":4, "CST":6, "CDT":5, "MST":7, "MDT":6, "PST":8, "PDT":7 }

# @markdown <br><font color="blue"><h2>Define time interval in minutes between plots</h2></font>
time_step = "5" # @param [5, 10, 15, 30, 45, 60]
time_step = int(time_step)

first_plot_time = f'{start_date}_{start_hour}{start_minute}'
last_plot_time = f'{end_date}_{end_hour}{end_minute}'

if first_plot_time >= last_plot_time:
  print("... Your start date/time must be before your end date/time !! ...")

# @markdown <br><font color="blue"><h2>Define lat/lon boundaries to plot</h2></font>
lat_min = 41 # @param {type:"raw"}
lat_max = 44 # @param {type:"raw"}
lon_min = -90 # @param {type:"raw"}
lon_max = -84 # @param {type:"raw"}

def make_float(val):
  try:
    return float(val)
  except:
    print("...... WARNING !! ......................\n")
    print(f"Can't convert {val} to float!\n")
    print("...... Check values and try again ......\n")
    return

lat_min = make_float(lat_min)
lat_max = make_float(lat_max)
lon_min = make_float(lon_min)
lon_max = make_float(lon_max)

# @markdown <br><font color="blue"><h2>Pick which features to include in the
# @markdown plot</h2></font>
plot_counties = True # @param {type:"boolean"}
plot_roads = False # @param {type:"boolean"}
# @markdown <font color="blue"><h5>Note: you'll first need a cities dictionary
# @markdown for your area if selecting plot_cities...</h5></font>
plot_cities = True # @param {type:"boolean"}

# @markdown <br><font color="blue"><h2>Check box for a "snow-friendly"
# @markdown colormap</h2></font>

snow_colormap = True # @param {type:"boolean"}
if snow_colormap:
  cmap_type = "winter"
else:
  cmap_type = "wdtd"

plot_key = f'{file_type}_{cmap_type}'
plot_dict = {'n0r_wdtd':{'cmap':'z_lowres','vmin':-30, 'vmax': 75, 'mult':5, 'subt':30.0},
             'n0q_wdtd':{'cmap':'wdtd_z','vmin':-32.5, 'vmax': 90, 'mult':0.5, 'subt':32.5},
             'n0r_winter':{'cmap':'winter_z','vmin':-30, 'vmax': 75, 'mult':5, 'subt':30.0},
             'n0q_winter':{'cmap':'winter_z','vmin':-32.5, 'vmax': 90, 'mult':0.5, 'subt':32.5}
             }

plot_values = plot_dict[plot_key]

# @markdown <br><font color="blue"><h2>If needed, adjust for undesired map stretching</h2>
# @markdown <i>This can be addressed in one or more of the following ways:</i>
# @markdown <font color="blue"><ol><li>Adjust your lat/lon boundary ranges</li>
# @markdown <li>Adjust the height and width dimensions of the figure.</li>
# @markdown <li>Adjust the plot aspect ratio ... >1 shrinks horizontal scale and
# @markdown vice versa.</li></ol></h4>

fig_height = 8 # @param {type:"raw"}
fig_height = int(fig_height)
fig_width = 5 # @param {type:"raw"}
fig_width = int(fig_width)
plot_aspect_ratio = 1.25 # @param {type:"raw"}
plot_aspect_ratio = float(plot_aspect_ratio)

# @markdown <br><font color="blue"><h2>Check to create animated gif</h2>
# @markdown <i>This will take time to load the software package ... creates a file called "animation.gif" </i></h4>
create_animation = False # @param {type:"boolean"}
# @markdown <h4><font color="blue"><i>This sets time between frames in hundredths of a second</i></font></h4>
animation_speed ="10" # @param [5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]
animation_speed = int(animation_speed)

cities = {
'Grand Rapids':{'lat':42.963,'lon':-85.668, 'lat_offset':0.04, 'lon_offset':0.06},
'Muskegon':{'lat':43.234,'lon':-86.248, 'lat_offset':0.04, 'lon_offset':0.06},
'Holland':{'lat':42.7875,'lon':-86.1089, 'lat_offset':0.04, 'lon_offset':0.06},
'South Haven':{'lat':42.4031,'lon':-86.2736, 'lat_offset':0.04, 'lon_offset':0.06},
'Lansing':{'lat':42.7325,'lon':-84.5556, 'lat_offset':0.00, 'lon_offset':0.06},
'Kalamazoo':{'lat':42.292,'lon':-85.587, 'lat_offset':-0.10, 'lon_offset':0.06},
'Ludington':{'lat':43.955,'lon':-86.4525, 'lat_offset':0.04, 'lon_offset':0.06},
'Big Rapids':{'lat':43.698,'lon':-85.4836, 'lat_offset':0.04, 'lon_offset':0.06},
'Mount Pleasant':{'lat':43.5978,'lon':-84.7675, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC109':{'lat':43.0534,'lon':-85.9080, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC158':{'lat':43.0923,'lon':-86.0977, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC159':{'lat':43.4101,'lon':-86.3146, 'lat_offset':0.06, 'lon_offset':0.06},
           }


In [2]:
# @title If you want to customize your own cities dictionary, click "Show code" below

#   You'll want to test the best offset position for each of the labels.
#   Positive "lat_offset" shifts the label north by that many degrees
#   Negative "lon_offest" shifts the label east by that many degrees

cities = {
'Grand Rapids':{'lat':42.963,'lon':-85.668, 'lat_offset':0.04, 'lon_offset':0.06},
'Muskegon':{'lat':43.234,'lon':-86.248, 'lat_offset':0.04, 'lon_offset':0.06},
'Holland':{'lat':42.7875,'lon':-86.1089, 'lat_offset':0.04, 'lon_offset':0.06},
'South Haven':{'lat':42.4031,'lon':-86.2736, 'lat_offset':0.04, 'lon_offset':0.06},
'Lansing':{'lat':42.7325,'lon':-84.5556, 'lat_offset':0.00, 'lon_offset':0.06},
'Kalamazoo':{'lat':42.292,'lon':-85.587, 'lat_offset':-0.10, 'lon_offset':0.06},
'Ludington':{'lat':43.955,'lon':-86.4525, 'lat_offset':0.04, 'lon_offset':0.06},
'Big Rapids':{'lat':43.698,'lon':-85.4836, 'lat_offset':0.04, 'lon_offset':0.06},
'Mount Pleasant':{'lat':43.5978,'lon':-84.7675, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC109':{'lat':43.0534,'lon':-85.9080, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC158':{'lat':43.0923,'lon':-86.0977, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC159':{'lat':43.4101,'lon':-86.3146, 'lat_offset':0.06, 'lon_offset':0.06},
           }


In [4]:
# @title Install the needed GIS software
!pip install rasterio &> /dev/null
!pip install cartopy &> /dev/null

In [7]:
# @title Ensure colormaps and GIS files are ready
import sys
import os
import subprocess
import glob
from os.path import exists
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import urllib.request
import requests
from bs4 import BeautifulSoup
import zipfile
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.io.shapereader as shpreader
import xarray as xr
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

os.chdir('/content')

def create_and_register_cmap(cmap_name, original_colors, position, rgb_input=True):
    """
    1) Converts list of color values from 8-bit (0-255 scale)
       to 0-1 decimal scale.

    2) The created final_color_list is combine with position list
       to create a matplotlib colormap.

    3) Registers the new colormap using the supplied cmap_name.

    Input
    ----------
             cmap_name : string
                         Name of the cmap you want to use and register

      original_ colors : list
                         8-bit tuples containing RGB values
                         example: [(127,256,192), (125,125,125)]

              position : list
                         node locations for color break points on
                         a 0 to 1 scale

            rgb_input : boolean
                        True: Color conversion from RGB to performed.
                       False: Color conversion from RGB is skipped.

    Returns
    -------
    None

    """
    if rgb_input:
        final_color_list = []
        for color in original_colors:
            converted_color_group = []
            for original_channel_value in color:
                new_channel_value = float(original_channel_value)/255
                converted_color_group.append(float(format(new_channel_value, '.3f')))
            converted_color_tuple = tuple(converted_color_group)
            final_color_list.append(converted_color_tuple)
    else:
        final_color_list = original_colors

    this_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(position, final_color_list)))
    mpl.colormaps.register(cmap=this_cmap, force=True)
    return


###############################################################################

winter_rgbs = [(0, 0, 0), (70, 70, 70), (210, 210, 210),
(0, 120, 255), (0, 0, 60), (225, 225, 0), (220, 100, 0), (220, 100, 0)]

winter_nodes = [0.0, 0.20, 0.38, 0.42, 0.48, 0.52, 0.60, 1.0]

create_and_register_cmap('winter_z', winter_rgbs, winter_nodes)

#------------------------------------------------------------------------------

wdtd_z_rgbs =[ (0, 0, 0), (0, 0, 0), (0, 0, 0), (130, 130, 130),
 (95, 189, 207), (57, 201, 105), (57, 201, 105), (0, 40, 0),
 (9, 94, 9), (255, 207, 0), (255, 207, 0), (255, 133, 0),
 (255, 0, 0), (89, 0, 0), (255, 245, 255), (225, 11, 227),
 (164, 0, 247), (99, 0, 214), (5, 221, 224), (58, 103, 181),
 (255, 255, 255),(255, 255, 255)]

def create_nodes(refs,denom,below_zero):
  nodes = []
  for r in refs:
    ref = (r + below_zero)/denom
    nodes.append(ref)
  return nodes

refs = [-32, -25.1, -25,14.99, 15,19.99, 20,34.99, 35,39.99, 40,49.99,
        50,59.99, 60,69.99, 70,74.99, 75,79.99, 80,90]

wdtd_z_nodes = create_nodes(refs,122,32)

refs = [-30, -25.1, -25, 14.99, 15,19.99, 20,34.99, 35,39.99, 40,49.99,
        50,59.99, 60,69.99, 70,74.99, 75]

low_nodes = create_nodes(refs,105,30)

create_and_register_cmap('wdtd_z', wdtd_z_rgbs, wdtd_z_nodes)
create_and_register_cmap('z_lowres', wdtd_z_rgbs[:-3], low_nodes)


###############################################################################

#scan this webpage to determine the most recent NWS County shapefile available
url = "https://www.weather.gov/gis/Counties"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
zip_links = soup.find_all("a", href=lambda href: href.endswith(".zip"))
if zip_links:
    zip_url = zip_links[-1]["href"]
    counties_url = os.path.join('https://www.weather.gov',str(zip_url)[1:])
    zip_file_name = counties_url.split("/")[-1]
    zip_file_root = zip_file_name[:-4]
    shapefile_name = f'{zip_file_root}.shp'
else:
    print("No zip files found on the website.")

if not exists(zip_file_name):
  urllib.request.urlretrieve(counties_url, "counties.zip")
  with zipfile.ZipFile('counties.zip', 'r') as zip_ref:
      zip_ref.extractall('.')

try:
  COUNTY_SHAPEFILE
except NameError:
  #reader = shpreader.Reader("tl_2019_us_county.shp")
  reader = shpreader.Reader(shapefile_name)
  features = list(reader.geometries())
  COUNTY_SHAPEFILE = cfeature.ShapelyFeature(features, ccrs.PlateCarree())

# download and read road shapefile only if plot_roads=True
if plot_roads:
  road_shapefile_name = "tl_2019_us_primaryroads.shp"
  road_url = "https://www2.census.gov/geo/tiger/TIGER2019/PRIMARYROADS/tl_2019_us_primaryroads.zip"
  if not exists(road_shapefile_name):
    urllib.request.urlretrieve(road_url, "roads.zip")
    with zipfile.ZipFile('roads.zip', 'r') as zip_roads:
      zip_roads.extractall('.')
  try:
    ROAD_SHAPEFILE
  except NameError:
    #reader = shpreader.Reader("tl_2019_us_primary_roads.shp")
    road_reader = shpreader.Reader(road_shapefile_name)
    features = list(road_reader.geometries())
    ROAD_SHAPEFILE = cfeature.ShapelyFeature(features, ccrs.PlateCarree())



plt.rcParams['font.size'] = 10
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['lines.markersize'] = 6
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['text.color'] = 'white'
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['scatter.marker'] = 'o'




In [ ]:
# @title Create your images!
import matplotlib as mpl

def create_valid_times(initial_time,final_time):

  start_time = datetime.strptime(initial_time, "%Y%m%d_%H%M")
  end_time = datetime.strptime(final_time, "%Y%m%d_%H%M")

  valid_times = []
  i = 0
  this_time = start_time + timedelta(minutes=time_step*i)
  valid_times.append(this_time)
  while this_time < end_time:
    i += 1
    this_time = start_time + timedelta(minutes=time_step*i)
    valid_times.append(this_time)

  return valid_times

valid_times = create_valid_times(first_plot_time,last_plot_time)


# ------------- Download pngs, create plots, save images -------------

URL_BASE = "https://mesonet.agron.iastate.edu/archive/data/"

os.chdir('/content')
# create fresh images directory to store new images to
result = subprocess.run(["rm -rf images ; mkdir images"], shell=True, capture_output=True, text=True)
fout = open('output.txt', 'w')
fout.write(f'dts,station,ref\n')
for plot_time in valid_times:
  local_time = plot_time - timedelta(hours=time_zone_dict[time_zone])

  file_dirs = datetime.strftime(plot_time,"%Y/%m/%d/GIS/uscomp/")
  file_timestamp = datetime.strftime(plot_time,"_%Y%m%d%H%M")
  url_path = f'{URL_BASE}{file_dirs}'
  file_root = f'{file_type}{file_timestamp}'
  downloaded_png = f'{file_root}.png'
  png_url = f'{url_path}{file_root}.png'
  wld_url = f'{url_path}{file_root}.wld'
  if not exists(downloaded_png):
    map = urllib.request.urlretrieve(png_url,downloaded_png)

  with rasterio.open(downloaded_png) as src:
    # Get the image data and metadata
    image_data = src.read()

    #slice (x,y) cartesian array out of current (1,x,y) array
    image_new = image_data[0,:,:]

    # convert 8-bit (0-255) values to reflectivity values as described at
    # https://mesonet.agron.iastate.edu/GIS/rasters.php?rid=2

    image_new = (image_new * plot_values['mult']) - plot_values['subt']


    #flip data vertically to move origin from upper left to lower left
    image_flipped = np.flipud(image_new)

    # determine images dimensions
    xlen = np.shape(image_flipped)[1]
    ylen = np.shape(image_flipped)[0]

    # create list of lat/lon values based on dimension lengths
    x = list(np.linspace(-126.0, -65.0, num=xlen, retstep=True)[0])
    y = list(np.linspace(23.0, 50.0, num=ylen, retstep=True)[0])

    # Create xarray add latitude/longitude dimensions using lists above
    coords = {'longitude': x, 'latitude': y}
    data = xr.DataArray(image_flipped, coords=coords, dims=['latitude', 'longitude'])
    data.attrs["units"] = "dBZ"
    data.attrs["long_name"] = "Reflectivity"

    # Determine region you want to plot
    subset = data.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))

    fig, ax = plt.subplots(1,1,figsize=(fig_height,fig_width),layout='constrained',subplot_kw={'projection': ccrs.PlateCarree()})
    subset.plot(ax=ax, cmap=plot_values['cmap'], vmin=plot_values['vmin'], vmax=plot_values['vmax'])


    ax.set_aspect(plot_aspect_ratio)
    if plot_counties:
      ax.add_feature(COUNTY_SHAPEFILE, facecolor='none', edgecolor='#cccccc', linewidth=0.7)

    if plot_roads:
      ax.add_feature(ROAD_SHAPEFILE, facecolor='none', edgecolor='cyan', linewidth=0.5)
    ax.add_feature(cfeature.STATES, facecolor='none', edgecolor='white', linewidth=1.3)
    ax.set_title(datetime.strftime(local_time,f'%b %d, %Y -- %I:%M %p {time_zone}'), color='black')
    if time_zone == 'UTC':
      ax.set_title(datetime.strftime(local_time,f'%b %d, %Y -- %H:%M {time_zone}'), color='black')
    #plt.tight_layout()
    target_fname = f'images/cropped_{downloaded_png}'
    if plot_cities:
      for key in cities:
        c = cities[key]
        lat,lon = c['lat'], c['lon']
        if ((lat > lat_min) & (lat < lat_max) & (lon > lon_min) & (lon < lon_max)):
          if 'MC' in key:
            plt.plot([lon], [lat], color='#FFAAAA', linewidth=1, marker='o', markersize=4, zorder=10)
            plt.text(lon + c['lon_offset'], lat + c['lat_offset'], key, color='#FFAAAA', zorder=11)
          else:
            plt.plot([lon], [lat], color='#FFFF00', linewidth=1, marker='o', markersize=6, zorder=10)
            plt.plot([lon], [lat], color='white', linewidth=1, marker='o', markersize=4, zorder=10)
            plt.text(lon + c['lon_offset'], lat + c['lat_offset'], key, color='#FFFFFF', zorder=11)
    plt.savefig(target_fname,bbox_inches='tight')


import glob
import os

if create_animation:
  !apt install imagemagick &> /dev/null
  os.chdir('/content/images')
  cmd = f'convert -delay {animation_speed} -loop 0 *.png animation.gif'
  os.system(cmd)

# tar images files together so you can download single file
os.chdir('/content/')
with zipfile.ZipFile('radar_images.zip', 'w') as f:
    for file in glob.glob('images/*'):
        f.write(file)


In [ ]:
!rm /content/*png
    'MC109':{'lat':43.0534,'lon':-85.9080, 'lat_offset':0.06, 'lon_offset':0.06},
    'MC158':{'lat':43.0923,'lon':-86.0977, 'lat_offset':0.06, 'lon_offset':0.06},
    'MC159':{'lat':43.4101,'lon':-86.3146, 'lat_offset':0.06, 'lon_offset':0.06}

x = np.arange(-126.0, -65.0, 0.005)
y = np.arange(23.0, 50.0, 0.005)
coords = {'longitude': x, 'latitude': y}

def extract_wld_data(wld_filename):
  req = urllib.request.Request(wld_filename)
  with urllib.request.urlopen(req) as resp:
    data = str(resp.read(), encoding='utf-8')
    values = data.split('\n')
    dx = float(values[0])
    dy = -1.0 * float(values[3])
    x = np.arange(-126.0, -65.0, dx)
    y = np.arange(23.0, 50.0, dy)
    coords = {'longitude': x, 'latitude': y}
    print(dx,dy,(len(x),len(y)))

  return coords


  dx = 0.01
  dts = datetime.strftime(plot_time,"%Y-%m-%d %H:%M:%SZ")
  for stn in ('MC109','MC158','MC159'):
    lat = cities[stn]['lat']
    lon = cities[stn]['lon']
    subset = data.sel(latitude=slice(lat - dx, lat + dx), longitude=slice(lon - dx, lon + dx))
    ref = np.mean(subset.data)
    fout.write(f'{dts},{stn},{ref}\n')

fout.close()

# @title Ignore this code
def map_ref(z):
  z_new = ((z + 32) / 122)
  return r

mc109 = df[df['station'] == 'MC109']['ref']
#print(list(mc109.index))

mc158 = list(df[df['station'] == 'MC158']['ref'])
mc159 = list(df[df['station'] == 'MC159']['ref'])

d = {'MC109': mc109, 'MC158': mc158, 'MC159': mc159}
mean_ref = pd.DataFrame(data=d, index=mc109.index)
#print(mean_ref)
plot = mean_ref.plot()


import pandas as pd
#print(data.data)
df = pd.read_csv('output.txt')
df['date'] = pd.DatetimeIndex(df['dts'])
df.set_index('date', inplace=True)
df.drop('dts',axis=1,inplace=True)
print(df.index)

mean_ref.to_csv('ref.csv', index=True)

cmap_dict = {}
with open('TWIP_BR_test.pal', 'r') as fin:
  for line in fin:
    if 'Color' in line:
      values = line.split()
      ref = float(values[1])
      norm_ref = map_ref(ref)
      rgb_slice = [int(c) for c in values[2:5]]
      rgb_tuple = tuple(rgb_slice)
      cmap_dict[norm_ref] = rgb_tuple

sorted_keys = sorted(list(cmap_dict.keys()))
sorted_rgbs = [cmap_dict[i] for i in sorted_keys]
print(sorted_rgbs)
print(sorted_keys)